## Trump Campaign Donations by City/State
## Author: Nick Silva
## DSO 545 Final Project

In [1]:
## The purpose of this notebook is to look at contributions over 2019 and analyze if Trump campaign rallies cause a spike
## in contributions in the state and city of the rally.
## This notebook uses the csv file "rallies.csv" from the Rally_Web_Scrape notebook for the rally locations

## The purpose of this notebook is to also learn how to use interactive widgets by creating drop down menus to select which
## state/city to look at. 

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

## import interactive packages
import ipywidgets as wg
from IPython.display import display
from matplotlib.dates import DateFormatter

In [3]:
## Read in trump csv
trump_ind = pd.read_csv("trump_ind.csv")
trump_ind = trump_ind.iloc[:, 1:]
trump_ind.head()

A:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (5,10,25,26,27,29) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,committee_name,report_year,report_type,image_number,line_number,transaction_id,file_number,entity_type,entity_type_desc,contributor_prefix,...,candidate_office_district,donor_committee_name,election_type,fec_election_type_desc,fec_election_year,is_individual,two_year_transaction_period,sub_id,pdf_url,line_number_label
0,"DONALD J. TRUMP FOR PRESIDENT, INC.",2017,Q1,201707209066951292,17A,SA17A.237312,1174081,IND,INDIVIDUAL,NaN,...,NaN,NaN,P2020,PRIMARY,2020.0,t,2018,4072120171426461530,http://docquery.fec.gov/cgi-bin/fecimg/?201707...,Contributions From Individuals/Persons Other T...
1,"DONALD J. TRUMP FOR PRESIDENT, INC.",2017,Q1,201707209066951320,17A,SA17A.232413,1174081,IND,INDIVIDUAL,NaN,...,NaN,NaN,P2020,PRIMARY,2020.0,t,2018,4072120171426461615,http://docquery.fec.gov/cgi-bin/fecimg/?201707...,Contributions From Individuals/Persons Other T...
2,"DONALD J. TRUMP FOR PRESIDENT, INC.",2017,Q1,201707209066951322,17A,SA17A.427279,1174081,IND,INDIVIDUAL,NaN,...,NaN,NaN,P2020,PRIMARY,2020.0,t,2018,4072120171426461620,http://docquery.fec.gov/cgi-bin/fecimg/?201707...,Contributions From Individuals/Persons Other T...
3,"DONALD J. TRUMP FOR PRESIDENT, INC.",2017,Q1,201707209066951322,17A,SA17A.445264,1174081,IND,INDIVIDUAL,NaN,...,NaN,NaN,G2016,GENERAL,2016.0,t,2018,4072120171426461621,http://docquery.fec.gov/cgi-bin/fecimg/?201707...,Contributions From Individuals/Persons Other T...
4,"DONALD J. TRUMP FOR PRESIDENT, INC.",2017,Q1,201707209066951322,17A,SA17A.446031,1174081,IND,INDIVIDUAL,NaN,...,NaN,NaN,P2020,PRIMARY,2020.0,t,2018,4072120171426461622,http://docquery.fec.gov/cgi-bin/fecimg/?201707...,Contributions From Individuals/Persons Other T...


In [4]:
## Read in the rallies csv
rallies = pd.read_csv("rallies.csv")
rallies = rallies.iloc[:, 1:]
rallies

,Date,City,State
0,2019-02-11,El Paso,TX
1,2019-03-28,Grand Rapids,MI
2,2019-04-27,Green Bay,WI
3,2019-05-08,Panama City Beach,FL
4,2019-05-20,Montoursville,PA
5,2019-06-18,Orlando,FL
6,2019-07-17,Greenville,NC
7,2019-08-01,Cincinnati,OH
8,2019-08-15,Manchester,NH
9,2019-09-09,Fayetteville,NC


In [5]:
##Get count of contributions by state in a dataframe
## filter for only positive contributions (no refunds) in 2019
## filter rally dataframe for only when we have contribution data until
states = trump_ind.loc[(trump_ind["contribution_receipt_date"] > '2018-12-31') & (trump_ind["contribution_receipt_amount"] > 0), ["contribution_receipt_date", "contribution_receipt_amount","contributor_city", "contributor_state", "contributor_zip"]]
max_date = states["contribution_receipt_date"].max()
rallies = rallies.loc[rallies.Date < max_date, :]
rallies["Date"] = pd.to_datetime(rallies["Date"])

##Rename columns/ format city names/ change date to datetime
states.columns = ["date", "amount", "city", "state", "zip"]
states["city"] = states["city"].str.title()
states.date = pd.to_datetime(states.date)
states.head()

A:\Anaconda\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


,date,amount,city,state,zip
374444,2019-05-10,685.25,Bolton Landing,NY,12814.0
374445,2019-05-10,37.50,Bolton Landing,NY,12814.0
374446,2019-05-10,1414.75,Bolton Landing,NY,12814.0
374447,2019-05-10,26.25,Bolton Landing,NY,12814.0
374448,2019-05-10,37.50,Bolton Landing,NY,12814.0


In [6]:
##Create a data dictionary to hold all zips within a 50 mile radius of each city that hosted a rally

zip_dict = {'Cincinnati' : [45220,45219,45225,45217,45229,45235,45223,45232,45254,45214,45206,45207,45202,45203,45224,45201,45221,45222,45234,45250,45253,45258,45262,45263,45264,45267,45268,45269,45270,45271,45273,45274,45277,45280,45296,45298,45299,45999,45212,41073,41074,45205,41072,45211,45216,41012,41019,41016,45208,45239,45237,45204,45209,45238,41071,41011,45213,41014,41075,45226,45215,45051,45231,45236,45248,45227,45218,45233,41025,41099,45247,45230,41017,41021,45240,45251,45243,41085,45246,41018,45001,41076,41048,45244,45252,45275,45041,45241,45174,45255,45242,41015,41022,45002,45147,45052,45111,45014,45030,45033,45249,45061,41042,41059,45245,45071,45069,45015,41051,41005,45150,45063,45140,41001,45012,45018,41080,45040,41094,41091,45011,45102,45039,41053,41063,47025,45053,45013,45103,45160,45034,41007,45055,45157,45044,47060,45050,45122,45062,45065,45067,45158,47035,47022,47001,41092,41030,47016,45162,45056,45152,45004,41006,45153,45036,45176,41033,47040,47038,45042,47032,45156,45005,45064,47012,47019,47010,45106,45070,47020,47041,45118,45112,45107,45120,41095,45066,47018,45130,45003,41035,45054,45154,47003,47031,45113,45119,41043,45311,45327,41046,45330,41097,41040,47039,47011,47021,45148,41052,45068,45342,47033,45343,45458,45032,41086,45121,47030,45114,47017,47006,45131,45449,45325,47043,45142,47353,45171,41054,47042,45459,45381,41004,41045,45370,45146,41002,47024,45345,45439,45320,45429,47036,45155,40355,47224,45305,47037,45440,45177,41083,45419,47034,45409,41061,45417,45420,41062,41010,47324,47263,41003,45437,47325,45168,45423,45469,45479,41044,45422,45402,45430,45401,45412,45413,45428,45470,45475,45481,45482,45490,41034,45410,45441,45448,45159,45426,47023,45406,45167,41098,45301,45405,45434,45403,45432,45416,45338,45309,47331,40359,47225,45697,45404,45164,45431,45385,45115,41008,45415,45435,45315,45414],
            'El Paso' : [79903,79905,79910,79930,79920,79929,79901,79902,79911,79913,79914,79917,79923,79926,79931,79937,79940,79941,79942,79943,79944,79945,79946,79947,79948,79949,79950,79951,79952,79953,79954,79955,79958,79960,79961,79976,79978,79980,79995,79996,79997,79998,79999,88510,88511,88512,88513,88514,88515,88517,88518,88519,88520,88521,88523,88524,88525,88526,88527,88528,88529,88530,88531,88532,88533,88534,88535,88536,88538,88539,88540,88541,88542,88543,88544,88545,88546,88547,88548,88549,88550,88553,88554,88555,88556,88557,88558,88559,88560,88561,88562,88563,88565,88566,88567,88568,88569,88570,88571,88572,88573,88574,88575,88576,88577,88578,88579,88580,88581,88582,88583,88584,88585,88586,88587,88588,88589,88590,88595,79968,79906,79925,79904,79918,79915,79922,79935,79912,88063,79924,79907,79936,79934,79932,88008,79835,79927,79908,79821,79916,79849,88081,88021,79928,88024,88027,79938,88072,88048,79836,79838,88058,79853,88044,88047,88002,88011,88003,88001,88046,88004,88006,88013,88342,88033,88032,88052,88005],
            'Fayetteville' : [28301,28302,28309,28305,28303,28331,28304,28311,28314,28307,28395,28306,28348,28308,28391,28312,28390,28371,28356,28357,28318,28342,28310,28344,28378,28323,27546,28384,28386,28339,28376,28385,28394,28392,28382,28399,28326,28368,28335,27543,28377,27552,28334,27505,27506,27332,28355,28350,27521,28387,28388,28332,28360,28372,27501,28328,28337,28359,28366,28329,28396,28315,27504,27331,28441,28358,27526,28374,28370,28364,28373,27330,28320,28327,27592,27540,28347,27524,28353,27376,27237,28352,27559,27562,28393,28351,28433,28341,27256,28383,28375,28363,27577,28444,27539,27252,27281,27529,27603,28343,27502,28325,28369,27520,28438,28340,27259,27207,29570,27325,27242,28319,28345,28365,27518,27593,29547,27528,29567,27606,28398,28434,27568,28453,28367,27569,27511,28458,28338,27530,27527,29594,28380,27213,28447],
            'Grand Rapids' : [49503,49501,49502,49510,49514,49516,49518,49523,49530,49555,49560,49588,49507,49506,49515,49599,49505,49504,49528,49509,49525,49519,49546,49548,49508,49468,49534,49512,49429,49544,49355,49356,49357,49418,49301,49321,49306,49428,49435,49315,49351,49316,49427,49317,49430,49426,49331,49335,49341,49345,49401,49302,49403,49464,49404,49323,49314,49348,49318,49330,49319,49333,48815,49325,49311,48881,48809,49460,48887,49451,49448,49328,49303,49417,49419,49344,49343,49424,49423,49422,48838,49326,49058,48849,49415,49339,49070,49327,48865,49456,49035,49347,49409,48846,49434,49010,49444,48897,48834,49329,49442,49322,48885,49046,49080,49078,49337,49453,48884,49406,49408,48870,49441,48851,49443,49440,49073,48890,48875,48888,49412,49457,49050,49450,49413,48860,49060,48850,49336,49096,49445,48861,48886,49083,48845,48873,49425,48852,49004,49062,49416,49055,49026,48811,48894,49056,48818,49020,49012,49063,49349,48835,48829,49006,49009,49017,49007,49461,49003,49005,49019,49016,48822,49021,49346,48812,49041,48837,49090,49001,49008,49037,49053,48891,49077,49048,49074,49027,49463,48856,49310,48889,49437,49013,49071,48853,49018,48830,48813,49452],
            'Green Bay' : [54302,54305,54306,54307,54308,54324,54344,54301,54311,54303,54304,54229,54313,54115,54173,54155,54217,54208,54126,54180,54165,54141,54171,54227,54131,54205,54130,54216,54162,54101,54240,54215,54137,54228,54123,54140,54247,54113,54204,54110,54214,54913,54136,54106,54230,54127,54152,54213,54911,54201,54241,54153,54107,54915,54169,54154,54160,54912,54919,54129,54207,54952,54226,54111,54220,54139,54914,54170,54942,54221,54957,54245,54124,53088,54232,54944,54956,54235,53014,53042,54112,54933,53063,54166,54157,54931,53061,53062,54922,54929,54985,54961,54901,53015,54947,54161,54174,54969,54143,54903,54906,54927,54135,53020,54902,53049,54949,54904,54986,54940,53083,54128,54928,54159,54948,54950,54978,54983,54209],
            'Greenville' : [27858,27879,27837,28590,27835,27836,27833,28513,27834,28530,27884,27817,27811,28586,27828,28538,27889,27812,27827,28554,28526,27871,27861,28580,28501,28523,28527,27825,27814,27829,27819,27892,28502,28503,27888,27841,27852,27821,27865,27886,27846,27873,28504,27864,27840,27883,27808,28551,27857,28519,28561,28563,28564,27881,28585,27806,27860,28562,28560,28573,27843,28525,27815,27962,27534,27893,27801,27813,28509,28522,27830,27849,28529,27894,27895,27983,27822,27878,27863,27810,28515,27531,28578,28583,27802,27532,27533,27803,28587,27874,28508,27851,28572,27896,28552,27809,28333,28556,27804,28510,27872,28537,27847,28571,28555,27875,27970,27530,27805,27957,27891,28533,27880,27807,27542,28574,28546,28532,27868,27569],
            'Manchester' : [3101,3105,3108,3111,3102,3104,3109,3103,3110,3106,3045,3032,3053,3052,3034,3054,3046,3304,3041,3070,3031,3036,3040,3038,3275,3057,3077,3064,3281,3873,3087,3063,3061,3055,3037,3302,3305,3051,3073,3841,3082,3234,3044,3301,3049,3060,3043,3062,3819,3079,3229,3811,3076,3258,3826,3086,3033,3290,3042,3291,3261,3047,3848,3865,1832,3859,1827,3242,1826,3048,3442,1844,3084,1879,3858,3307,1463,3263,3303,1854,3833,1850,1831,1841,1842,1830,3861,3218,1840,1863,1853,3244,1835,3458,1860,3827,3856,3224,1843,1469,3884,1851,3825,1474,3071,1852,3815,1450,3857,3440,1470,1471,3449,3468,5501,1913,1886,1810,1885,1472,3225,1812,1899,1824,1834,1985,3885,1876,1431,3824,1845,1464,3823,1432,3278,3844,3268,3221,1862,3839,1462,1833,1921,3837,3461,3237,3843,3276,1460,3866,3874,3840,1822,3450,3444,3273,3842,3452,3820,1952,1922,1741,3862,1821,3867,1434,1950,1718,3835,1430,1865,3821,1887,3220,1420,3457,3822,3235,3464,3252,1866,3801,1864,3272,3260,1451,1969,1719,3280,1889,3878,1720,1951,1730,1949,3869,3871,3289,3809,1453,3870,3255,3216,1467,1803,3802,3803,3804,1983,3868,3455,3903,1805,1867,3855,1742,3810,3456,3231,1937,1523,1473,1475,3447,1731,3904,1938,1801,3854,3445,1440,1940,3465,3901,3851,1888,3269,1923,1813,1815,1775,1740,1754,3247,2420,3752,3287,1561,1880,1441,2421,3887,1773,1982,3257,3448,3233,3246,1936,3249,3905,3243,1960,1984,2180,4027,1890,3908,1438,3431,1564,3435,1436,1510,2479,1776,1749,3782,3909,2475,1961,2451,2176,3906,2476,2156,3230,1915,2474,1503,3605,1541,3911,1929,3446,1906,1904,2155,2452,1368,1905,3754,1970,1468,1971,2478,3253,3852,3607,1778,3896,2148,2454,3469,1965,3256,2153,2144,3298,3299,3751,2453,1902,2493,2140,3894,1452,1910,1752,3284,3602,3902,1903,1901,2472,2471,2477,3910,3601,2138,3773,1944,2145,2149,1945,1907,1505,2465,2466,3853,2143,2460,1701,2151,3470,2458,2163,2150,4001,4083,3222,1532,4073,1583,3872,2129,2456,3907,2462,1930,2238,2134,2495,1331,2139,2135,2141,1522,1908,2468,1703,1704,1705,2142,2222,2114,3604,2133,3240,2113,2446,3608,3462,4054,4090,2481,1772,2203,2109,2215,2108,2112,2117,2123,2137,2455,2196,2201,2204,2206,2211,2217,2241,2266,2283,2284,2293,2297,1931,1745,2128,2116,2110,2464,2199,2461,2115,2447,1966,2111,2459,2205,2482,2457,1760,1543,2467,2120,1702,3850,2494,2445,1364,2118,3603,1520,2210,3753,3467,1546,2298,1606,2152,3254,3226,4076,1378,2119],
            'Montoursville' : [17702,17810,17703,17705,17744,17701,17752,17720,17856,17728,17730,17740,17887,17777,17721,17754,17837,17779,17886,17749,17844,17748,17756,17883,17835,17747,17847,17771,17850,17772,17880,17726,17865,17829,17762,16872,17745,17855,17739,17889,17737,17776,17862,17884,17751,17845,17750,17763,17857,16882,17742,17885,17821,16930,17876,17843,18619,16820,17870,17765,17831,17868,17882,17833,16848,17842,16938,17822,17774,17861,17723,17813,17801,16854,17731,17846,17768,16852,16826,17877,17827,17812,17769,17727,17735,16832,16912,17758,17860,17086,16841,16822,18616,17853,17864,16939,18626,16911,17815,16875,16856,17724,17824,17881,17760,17014,17867,17823,17830,17878,17814,16864,17866,17063,17017,16926,16917,17841,16910,17049,17859,17764,17820,16823,17080,18611,17858,18614,18632,17832,17836,18833,16828,17834,17729,17094,16901,17872,17045,16853,18631,17076,16921,17005,18603,17851,16932,17941,17061,18622,17964,17840,18832,17023,17062,17978,17888,17048,16947,17030,18628,16829,16945,17968,17059,16933,17097,18655,17945,17983,18635,17920,17943,17099,17084,17009,17938,16827,17985,17058,17921,16835,16851,17056,17098,17778,17074,18601,18656,18814,17936,16914,18241,17935,16845,16874,16802,16801,16844,17946,18848,17044,16804,16805,17980,18246,17957,16922],
            'Orlando' : [32801,32802,32853,32854,32855,32856,32857,32858,32859,32860,32861,32862,32867,32868,32869,32872,32877,32878,32885,32886,32887,32891,32897,32896,32803,32805,32806,32804,32814,32789,32839,32807,32790,32793,32733,32812,32794,32811,32808,32809,32799,32751,32822,32810,32792,32835,32730,32715,32716,32818,32701,32817,32714,32819,32710,32707,34734,32825,32718,32719,34761,32824,32829,32816,32752,32791,32708,32827,32750,32837,34786,32826,32703,32704,32836,32828,32821,34777,34778,32762,32779,32765,32768,32831,32746,34743,32795,32712,32832,32830,34760,32773,34787,34741,34740,32820,34742,34745,34744,32798,32772,34756,32766,32747,32833,32757,32771,32776,34729,34747,33848,34770,34746,32732,34769,32777,34755,34711,34715,32745,32834,33896,32713,32753,34705,32709,32756,34712,34713,33858,32725,34758,32728,34771,32739,34714,32778,32736,33897,32763,32738,34772,34737,32774,32727,33837,32754,32726,33836,32764,32796,34736,34797,32706,32720,32784,32780,34753,32744,34759,34748,32767,32735,33845,32721,33868,32927,32723,34762,34773,34788,32775,32781,32783,32926,33844,32168,34749,34789,32702,33850,33851,32759,32722,32959,32141,33514,33881,32724,34731,32923,32924,32922,33838,32132,32955,32815,33877,32956,33585,33823,32158,32159,33882,33883,33888,32953,33885,32124,32130,32170,33809,32954,32169,32128,33884,32163,33898,32195,32105,32102,33839,33880,32162,34785,33597,32940,32133,33853,32127,33856,32129,32183,33805,32180,32119,32920,32952,33801,33854,33521,33849,32931,32899,32934,33859,33810,33537,32114,33513,32932,33840,32179,32904,33806,33802,34739],
            'Panama City Beach' : [32408,32407,32401,32406,32402,32411,32412,32417,32461,32405,32444,32409,32413,32403,32404,32437,32466,32462,32410,32459,32449,32438,32439,32430,32463,32428,32465,32457,32455,32456,32550,32427,32420,32335,32421,32435,32422,32541,32424,32578,32425,32540,32321,32448,32464,32431,32588,32320],
            'Rio Rancho' : [87144,87124,87004,87048,87113,87107,87109,87043,87120,87122,87104,87114,87110,87131,87193,87187,87101,87103,87197,87184,87190,87191,87192,87185,87195,87196,87198,87199,87153,87181,87176,87174,87158,87154,87125,87194,87151,87102,87111,87112,87106,87001,87119,87053,87108,87117,87121,87123,87008,87105,87116,87047,87052,87068,87072,87024,87041,87115,87022,87044,87059,87083,87010,87042,87015,87025,87507,87031,87060,87040,87023,87544,87056,87061,87007,87508,87540,87545,87026,87038,87002,87592,87594,87504,87503,87502,87509,87505,87506]}

In [7]:
def zip_counter(c):
    ## This function will by called by the city_plot function
    ## The purpose of this function is to create a df of the zips contained in the data dictionary for the selected state
    ## The function then groups these zips by date and returns a dataframe with the date and count of donations.
    
    zips = zip_dict[c]
    zips = pd.Series(zips)
    new_df = pd.DataFrame()
    final_df = pd.DataFrame()
    
    
    for i in np.arange(0,len(zips)):
        city_zip = states.loc[states.zip == zips[i], ["date", "zip"]]
        new_df = new_df.append(city_zip)

    final_df = new_df.groupby("date").count().reset_index()
    
    return(final_df)
    

In [8]:
def top_five(largest):
    ## the purpose of this function is to sort the dataframes containing the contribution counts, sort, and slice for 
    ## the 3 dates with the largest amount of contributions. The function then changes those dates into a string
    ## and returns this string to be plotted below both the state and city plots. 
    
    largest = largest
    largest = largest.loc[:, ["date", "zip"]]
    largest = largest.sort_values("zip", ascending = False)
    largest = largest.iloc[:5, 0]
    
    date1 = largest.iloc[0].strftime("%B %d %Y")
    date2 = largest.iloc[1].strftime("%B %d %Y")
    date3 = largest.iloc[2].strftime("%B %d %Y")
    date4 = largest.iloc[3].strftime("%B %d %Y")
    date5 = largest.iloc[4].strftime("%B %d %Y")
    top_5_dates = "Five largest contribution dates: \n" + date1 + "\n" + date2 + "\n" + date3 + "\n" + date4 + "\n" + date5
    
    
    
    return(top_5_dates)

In [9]:
def state_plot(abbrev):
    ## This function will be called by the interactive state widget

    ## The purpose of this function is to create the plot of total contributions over time filtered by the
    ## state selected in the dropdown 
 
    plt.figure(figsize = (15,8), dpi = 350)

    state = states.loc[states.state == abbrev, :]  
    
    contributions = state.groupby("date").count().reset_index()
    plt.plot(contributions.date, contributions.amount)

    ## create a dataframe of rally states to be used as verticle line
    rally_state = rallies.loc[rallies.State == abbrev, :] 
    
    ax = plt.gca()
    
    ## plot each rally using a verticle line - for loop is used for states with multiple rallies
    for i in np.arange(0,len(rally_state)):
        plt.axvline(x = rally_state.iloc[i,0],
                    linestyle = "--",
                    color = "r")
        
        ##get rally date for graph           
        rally_date = rally_state.iloc[i,0]
        rally_date.strftime("%b %d")
        
        plt.text(x = rally_state.iloc[i,0],
                 y = .85*(contributions.amount.max()),
                 s = "  " + str(rally_state.iloc[i,1]) + "\n" + "  " + rally_date.strftime("%b %d"))
    
    ## Add chart elements
    plt.title("2019 Contributions to Trump Campaign in " + abbrev)
    ax.set_ylabel("Count of Contributions")
    ax.set_xlabel("2019")
    
    ## Format x-axis
    date_form = DateFormatter("%B")
    ax.xaxis.set_major_formatter(date_form)
    
    ax.grid(axis = "y",
            linestyle = "--",
            alpha = 0.4)
    
    ## call function to get the 3 largest contributions
    largest = top_five(contributions)
    
    ## plot largest contributions below plot
    prop = dict(boxstyle='round',facecolor = 'white', alpha=0.2)
    ax.text(.4, -0.1, str(largest), transform=ax.transAxes, fontsize=10,
        verticalalignment='top', bbox=prop)
    
    
    
    plt.show()

In [10]:
def city_plot(c):
    plt.figure(figsize = (15,8), dpi = 350)
    
     
    
    ## call zip function and plot
    plot_me = zip_counter(c)
    
    plt.plot(plot_me.date, plot_me.zip)

    # create variable of rally to be used for verticle line
    rally_state = rallies.loc[rallies.City == c, :]
    
    ax = plt.gca()
    ## plot the rally as a vertical line
    for i in np.arange(0,len(rally_state)):  ##does this need to be a for loop?
        plt.axvline(x = rally_state.iloc[i,0],
                    linestyle = "--",
                    color = "r")
        
    ## add rally date to vertical line
    rally_date = rally_state.iloc[i,0]
    rally_date.strftime("%b %d")
        
    plt.text(x = rally_state.iloc[i,0], 
             y = .85*(plot_me.zip.max()),
             s = "  " + rally_date.strftime("%b %d"))
        
    ## Add chart elements
    plt.title("2019 Contributions to Trump Campaign in 50 mile radius of " + c)
    ax.set_ylabel("Count of Contributions")
    ax.set_xlabel("2019")
    
    ## Format x-axis
    date_form = DateFormatter("%B")
    ax.xaxis.set_major_formatter(date_form)
    
    ax.grid(axis = "y",
            linestyle = "--",
            alpha = 0.4)
    
    ## call function to get the 3 largest contributions
    largest = top_five(plot_me)
    
    ## plot largest contributions below plot
    prop = dict(boxstyle='round',facecolor = 'white', alpha=0.2)
    ax.text(.4, -0.1, str(largest), transform=ax.transAxes, fontsize=10,
        verticalalignment='top', bbox=prop)

    plt.show()

In [11]:
## Get df of unique rally states for dropdown list
options = rallies.State.unique()
options.sort()

## Create dropdown list
state_select = wg.Dropdown(
    options = options,
    value = options[0],
    description = "State:",
    disabled = False

)

In [12]:
## Get df of unique rally cites for dropdown list
options = rallies.City.unique()
options.sort()

## Create dropdown list
city_select = wg.Dropdown(
    options = options,
    value = options[0],
    description = "City:",
    disabled = False

)

In [13]:
##Create interaction for states plot

wg.interact(state_plot, abbrev = state_select)

interactive(children=(Dropdown(description='State:', options=('FL', 'MI', 'NC', 'NH', 'NM', 'OH', 'PA', 'TX', …

<function __main__.state_plot(abbrev)>

In [14]:
##Create interaction for city plot

wg.interact(city_plot, c = city_select)

interactive(children=(Dropdown(description='City:', options=('Cincinnati', 'El Paso', 'Fayetteville', 'Grand R…

<function __main__.city_plot(c)>